In [43]:
import os
from PIL import Image

import scipy
import numpy as np
import benfordslaw
import torch


benford = np.array([30.1, 17.6, 12.5, 9.7, 7.9, 6.7, 5.8, 5.1, 4.6]) / 100

def mlh(bin_percent_):
    return scipy.stats.pearsonr(benford, bin_percent_[1:])[0]

def bincount(tensor):
    counts = torch.zeros(10)
    for i in range(10):
        counts[i] = torch.count_nonzero(tensor == i)
    return counts

def bin_count(tensor):
    tensor = tensor.abs() * 1e10
    long_tensor = torch.log10(tensor).long()
    tensor = tensor // 10 ** long_tensor
    return bincount(tensor.long())

def bin_percent(tensor):
    return tensor / tensor.sum()

In [62]:
from torchvision import transforms

image_path = '/home/dodant/Documents/Datasets/CIFAR 10 - images/train'
# image_path = '/home/dodant/Documents/Datasets/CIFAR 100 - images/train_image'
# mean = [0.4913997551666284, 0.48215855929893703, 0.4465309133731618]
# std = [0.24703225141799082, 0.24348516474564, 0.26158783926049628]
    
for label in os.listdir(image_path):
    # print(label)
    img_list_jpg = [img for img in os.listdir(image_path + '/' + label) if img.endswith(".jpg")]
    
    img_list_np = []
    digit_count = torch.zeros(10)

    for i in img_list_jpg:
        img = Image.open(image_path + '/' + label +'/' + i)
        
        timg_ = torch.Tensor(np.array(img))
        
        # timg_ = transforms.Normalize(mean, std)(timg_.view(3,32,32))
        timg_ = transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))(timg_.view(3,32,32))
        digit_count += bin_count(timg_)

    print(f'{label} : {mlh(bin_percent(digit_count)):.3f}')

/tmp/ipykernel_148344/2462985517.py:24: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  tensor = tensor // 10 ** long_tensor


automobile : 0.912
frog : 0.936
ship : 0.766
horse : 0.894
bird : 0.865
deer : 0.909
airplane : 0.701
cat : 0.894
dog : 0.874
truck : 0.884


## Original
automobile : 0.945
frog : 0.885
ship : 0.936
horse : 0.924
bird : 0.902
deer : 0.872
airplane : 0.941
cat : 0.932
dog : 0.927
truck : 0.947

## Normalize (0.5, 0.5, 0.5), (0.5, 0.5, 0.5)
automobile : 0.912
frog : 0.936
ship : 0.766
horse : 0.894
bird : 0.865
deer : 0.909
airplane : 0.701
cat : 0.894
dog : 0.874
truck : 0.884

# Normalize
mean = [0.4913997551666284, 0.48215855929893703, 0.4465309133731618]
std = [0.24703225141799082, 0.24348516474564, 0.26158783926049628]
automobile : 0.819
frog : 0.525
ship : -0.021
horse : 0.527
bird : 0.259
deer : 0.332
airplane : -0.357
cat : 0.608
dog : 0.480
truck : 0.859